## Implementation

The Variational Autoencoder will now be implemented and its performance assessed using the MNIST digits and Fashion MNIST datasets.

In [1]:
import random
from collections import defaultdict
from typing import Tuple, Dict

import matplotlib.pyplot as plt
import torch
import torchvision
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm.notebook import tqdm

ACCELERATOR = 'cpu'

### Dataset

The Torchvision library provides access to the MNIST digits and Fashion MNIST datasets. For further processing, these datasets will be enveloped with transformations such as:
- Conversion from PIL image to torch tensor
- Standardization

The estimated mean for the MNIST dataset is 0.13, and the standard deviation is 0.31. These estimates can be readily verified by computing the mean and std across all pixels in the training dataset.

In [2]:
MNIST_MEAN = 0.13
MNIST_STD = 0.31

def standardize_transform(x: torch.Tensor) -> torch.Tensor:
    """
    Standardizes a given tensor using the precomputed MNIST mean and standard deviation.

    Args:
        x: Input tensor to standardize.

    Returns:
        Standardized tensor.
    """
    return (x - MNIST_MEAN) / MNIST_STD


def standardize_inverse(x: torch.Tensor) -> torch.Tensor:
    """
    Reverses the standardization on a tensor using the precomputed MNIST mean and standard deviation.

    Args:
        x: Input tensor to de-standardize.

    Returns:
        De-standardized tensor.
    """
    return x * MNIST_STD + MNIST_MEAN


class UnsupervisedMNIST(Dataset):
    """
    A class that wraps the MNIST dataset for unsupervised learning. It allows to work with either the Fashion MNIST
    or the original MNIST dataset. This dataset will be used for the Autoencoder and VAE training (in the unsupervised manner).
    """
    def __init__(self, train: bool = True, fashion: bool = False):
        """
        Initializes the dataset object, downloads the dataset and prepares the transform.

        Args:
            train: Whether to load the training split (True) or the test split (False).
            fashion: Whether to load the Fashion MNIST dataset (True) or the original MNIST dataset (False).
        """
        cls = torchvision.datasets.FashionMNIST if fashion else torchvision.datasets.MNIST
        self._mnist = cls(root='/tmp/mnist', train=train, download=True)
        self._to_tensor = transforms.ToTensor()

    def __getitem__(self, i: int) -> Tuple[torch.Tensor, torch.Tensor]:
        raw_img, _ = self._mnist[i]
        img = self._to_tensor(raw_img)
        return standardize_transform(img)

    def __len__(self) -> int:
        return len(self._mnist)

    def label(self, i: int) -> int:
        """
        Returns the label of an image in the dataset at a given index.
        This is only required for the future visualizations.

        Args:
            i: The index of the image for which to return the label.

        Returns:
            The label of the image at the given index.
        """
        return self._mnist[i][1]
    
    
mnist_example = UnsupervisedMNIST(train=True)
mnist_example[0].shape, mnist_example.label(0)

(torch.Size([1, 28, 28]), 5)

### Implementation of the AutoEncoder neural network model

In this section, an Autoencoder neural network is assembled. This is followed by the development of a Variational Autoencoder, which employs a similar structure. Although a latent space of two dimensions may not accurately represent the entirety of the MNIST dataset, it is utilized here for the simplicity of visualization.

In [3]:
class AutoEncoder(nn.Module):
    """
    Implementation of AutoEncoder (MLP).
    MLP might be more efficient than CNN for very shallow networks (just like in this case).
    """
    def __init__(self, latent_dim: int = 2):
        """
        Args:
            latent_dim: Latent dimension
        """
        super().__init__()

        # Expects input shape 1x28x28
        self._encoder = nn.Sequential(
            nn.Flatten(start_dim=-3),  # size: 28*28 = 784
            nn.Linear(784, 392),
            nn.ReLU(),
            nn.Linear(392, 196),
            nn.ReLU(),
            nn.Linear(196, latent_dim)
        )
        # encoder ~ posterior surrogate approximation

        self._decoder = nn.Sequential(
            nn.Linear(latent_dim, 196),
            nn.ReLU(),
            nn.Linear(196, 392),
            nn.ReLU(),
            nn.Linear(392, 784),
            nn.Unflatten(-1, (1, 28, 28))
        )
        # decoder - likelihood

    def show_encoder_transforms(self, x: torch.Tensor) -> None:
        """
        Prints list of encoder layers and output shapes. Useful for debugging and model validation.

        Args:
            x: Random input (with shape 1, 28, 28)
        """
        print('---Encoder transforms---')
        print(f'Input shape: {x.shape}')
        for i, layer in enumerate(self._encoder):
            layer_name = type(layer).__name__
            x = layer(x)
            print(f'[{i+1}] name={layer_name}, shape={x.shape}')

    def show_decoder_transforms(self, x: torch.Tensor) -> None:
        """
        Prints list of encoder layers and output shapes. Useful for debugging and model validation.

        Args:
            x: Random input (with shape 1, 28, 28)
        """
        print('---Decoder transforms---')
        print(f'[0] Input shape: {x.shape}')
        for i, layer in enumerate(self._decoder):
            layer_name = type(layer).__name__
            x = layer(x)
            print(f'[{i+1}] name={layer_name}, shape={x.shape}')
        print()

    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        z = self._encoder(x)
        x_hat = self._decoder(z)
        return x_hat, z


ae = AutoEncoder()
ae.show_encoder_transforms(torch.randn(1, 28, 28, dtype=torch.float32))
ae.show_decoder_transforms(torch.randn(2, dtype=torch.float32))

---Encoder transforms---
Input shape: torch.Size([1, 28, 28])
[1] name=Flatten, shape=torch.Size([784])
[2] name=Linear, shape=torch.Size([392])
[3] name=ReLU, shape=torch.Size([392])
[4] name=Linear, shape=torch.Size([196])
[5] name=ReLU, shape=torch.Size([196])
[6] name=Linear, shape=torch.Size([2])
---Decoder transforms---
[0] Input shape: torch.Size([2])
[1] name=Linear, shape=torch.Size([196])
[2] name=ReLU, shape=torch.Size([196])
[3] name=Linear, shape=torch.Size([392])
[4] name=ReLU, shape=torch.Size([392])
[5] name=Linear, shape=torch.Size([784])
[6] name=Unflatten, shape=torch.Size([1, 28, 28])



#### Training Autoencoder

In [4]:
def train_autoencoder(
    model: AutoEncoder,
    train_dataset: Dataset,
    val_dataset: Dataset,
    epochs: int = 30,
    scheduler_step_size: int = 15,
    batch_size: int = 64,
    num_workers = 2,
    accelerator: str = 'cpu'
) -> dict:
    """
    Trains the AutoEncoder model on the provided datasets for a specified number of epochs.

    Args:
        model: An instance of AutoEncoder that will be trained.
        train_dataset: The dataset object used for training.
        val_dataset: The dataset object used for validation.
        epochs: The number of epochs to train the model. Defaults to 20.
        scheduler_step_size: Step size for learning rate scheduler. Defaults to 10.
        batch_size: The size of the batches for training and validation. Defaults to 64.
        num_workers: Number of workers for data loading. Defaults to 2.
        accelerator: The device on which to train the model. Defaults to 'cpu'.

    Returns:
        A dictionary containing the model's training and validation losses.
    """
    loss_func = nn.MSELoss()
    model.to(accelerator)
    optimizer = torch.optim.Adam(lr=1e-3, params=model.parameters())
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = scheduler_step_size)
    
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers)
    val_dataloader = DataLoader(val_dataset, batch_size, num_workers=num_workers, shuffle=True)
    
    history = defaultdict(list)
    
    for epoch in tqdm(range(1, epochs+1)):
        total_loss = 0.0
        n_steps = 0
        
        model.train()
        for img in train_dataloader:
            optimizer.zero_grad()
            
            img = img.to(accelerator)
            reconstructed_img, _ = model(img)
            
            loss = loss_func(img, reconstructed_img)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.detach().cpu().item()
            n_steps += 1
            
        scheduler.step()
            
        train_loss = total_loss / n_steps
        
        total_loss = 0.0
        n_steps = 0
        
        model.eval()
        with torch.no_grad():
            for img in val_dataloader:
                img = img.to(accelerator)
                reconstructed_img, _ = model(img)

                loss = loss_func(img, reconstructed_img)
                
                total_loss += loss.detach().cpu().item()
                n_steps += 1
                
        val_loss = total_loss / n_steps
        
        print(f'[Epoch={epoch:02d}]: train_loss={train_loss:.2f}, val_loss={val_loss:.2f}')
        
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['epoch'].append(epoch)
        
    return dict(history)
        

autoencoder = AutoEncoder()
history = train_autoencoder(
    model=autoencoder,
    train_dataset=UnsupervisedMNIST(train=True),
    val_dataset=UnsupervisedMNIST(train=False),
    accelerator=ACCELERATOR
)

  0%|          | 0/30 [00:00<?, ?it/s]

Traceback (most recent call last):
  File "<string>", line 1, in <module>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    exitcode = _main(fd, parent_sentinel)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'UnsupervisedMNIST' on <module '__main__' (built-in)>
    se

RuntimeError: DataLoader worker (pid(s) 20143, 20144) exited unexpectedly

#### Autoencoder demonstration

In this section autoencoder reconstruction and latent space will be showcased.

In [ ]:
@torch.no_grad()
def show_naive_autoencoder_error_analysis_visualization(
    model: AutoEncoder,
    val_dataset: Dataset,
    accelerator: str = 'cpu'
) -> None:
    """
    Displays a naive visualization for error analysis of an autoencoder.

    This function generates and displays a visual comparison of original and reconstructed
    images using a trained autoencoder model. The error analysis is considered naive because
    it only visually inspects a small percentage of the individual reconstruction errors.

    Args:
        model: The trained autoencoder model to use for image reconstruction.
        val_dataset: The dataset to use for generating the visualization. Typically,
            this should be a validation or test dataset.
        accelerator: The device to perform operations. Default is 'cpu'.
            Options are 'cpu' or 'cuda' if GPU is available.
    """
    N_SAMPLES = 6
    indices = list(range(len(val_dataset)))
    sampled_indices = random.choices(indices, k=N_SAMPLES)
    model.to(accelerator)
    
    _, axs = plt.subplots(figsize=(2 * N_SAMPLES + 1, 5), nrows=2, ncols=N_SAMPLES)
    for i in range(N_SAMPLES):
        index = sampled_indices[i]
        img = val_dataset[index]
        img = img.to(accelerator)
        img_hat, _ = model.forward(img)
        img, img_hat = [v.detach().cpu().numpy() for v in [img, img_hat]]
        
        axs[0][i].imshow(img[0])
        axs[1][i].imshow(img_hat[0])
        axs[0][i].axis('off')
        axs[1][i].axis('off')
           
    plt.show()

show_naive_autoencoder_error_analysis_visualization(
    model=autoencoder,
    val_dataset=UnsupervisedMNIST(train=False),
    accelerator=ACCELERATOR
)

The model appears to reconstruct the images reasonably well, though a certain level of blurriness is evident. Improvements in image sharpness might be achieved by expanding the latent dimension space (e.g. `latent_dim=8`).

In [ ]:
@torch.no_grad()
def show_ae_latent_space_distribution_visualization(
    model: AutoEncoder,
    val_dataset: Dataset,
    accelerator: str = 'cpu'
) -> None:
    """
    Displays the distribution of data in the Autoencoder' latent space.

    Args:
        model: The autoencoder model.
        val_dataset: Dataset used for the visualization.
        accelerator: The device for calculations ('cpu' or 'cuda').
    """
    COLORS = ['red', 'blue', 'green', 'yellow', 'orange', 'cyan', 'gray', 'black', 'purple', 'pink']
    model.eval()
    model.to(accelerator)
    
    zs = defaultdict(list)
    for i in tqdm(range(len(val_dataset))):
        img = val_dataset[i].to(accelerator)
        label = val_dataset.label(i)
        _, z = autoencoder(img)
        
        zs[label].append(z.detach().cpu())
        
    zs = {k: torch.stack(v) for k, v in zs.items()}
    zs = dict(sorted(zs.items()))
    plt.figure(figsize=(12, 8))
    for i, (label, z) in enumerate(zs.items()):
        z = z.numpy()
        plt.scatter(z[:, 0], z[:, 1], color=COLORS[i], label=str(label))
        
    plt.legend()
    plt.title('Latent Space')
    plt.xlabel('Z1')
    plt.ylabel('Z2')
    plt.show()
    
show_ae_latent_space_distribution_visualization(
    model=autoencoder,
    val_dataset=UnsupervisedMNIST(train=False),
    accelerator=ACCELERATOR
)

The overlapping class clusters and extensive spread in the latent space suggest potential underfitting of the model. Expanding the latent dimension could potentially result in better separation of clusters. However, this is not necessarily true for VAEs, where the KL component of the ELBO loss constrains clusters to be closer to the center.

## Variational Autoencoder

In this segment, the implementation of the Variational Autoencoder (VAE) model is carried out. Notably, the VAE model retains all capabilities of the Autoencoder (AE) model while also offering additional features such as:

- generating fresh images by sampling from the prior;
- creating new images influenced by an input, by sampling from the posterior;
- performing interpolation over the latent space.

In [ ]:
class VariationalAutoEncoder(nn.Module):
    """
    Implementation of VAE (MLP).
    MLP might be more efficient than CNN for very shallow networks (just like in this case).
    """
    def __init__(self, latent_dim: int = 2):
        """
        Args:
            latent_dim: Latent dimension
        """
        super().__init__()
        self._latent_dim = latent_dim

        self._encoder = nn.Sequential(
            nn.Flatten(start_dim=-3),  # size: 28*28 = 784
            nn.Linear(784, 392),
            nn.ReLU(),
            nn.Linear(392, 196),
            nn.ReLU(),
            nn.Linear(196, 2 * latent_dim)
        )
    
        self._decoder = nn.Sequential(
            nn.Linear(latent_dim, 196),
            nn.ReLU(),
            nn.Linear(196, 392),
            nn.ReLU(),
            nn.Linear(392, 784),
            nn.Unflatten(-1, (1, 28, 28))
        )
        
    def show_encoder_transforms(self, x: torch.Tensor) -> None:
        """
        Prints list of encoder layers and output shapes. Useful for debugging and model validation.

        Args:
            x: Random input (with shape 1, 28, 28)
        """
        print('---Encoder transforms---')
        print(f'Input shape: {x.shape}')
        for i, layer in enumerate(self._encoder):
            layer_name = type(layer).__name__
            x = layer(x)
            print(f'[{i+1}] name={layer_name}, shape={x.shape}')

    def show_decoder_transforms(self, x: torch.Tensor) -> None:
        """
        Prints list of encoder layers and output shapes. Useful for debugging and model validation.

        Args:
            x: Random input (with shape 1, 28, 28)
        """
        print('---Decoder transforms---')
        print(f'[0] Input shape: {x.shape}')
        for i, layer in enumerate(self._decoder):
            layer_name = type(layer).__name__
            x = layer(x)
            print(f'[{i+1}] name={layer_name}, shape={x.shape}')
        print()
            
    def forward(self, x: torch.Tensor, sample: bool = True) -> Dict[str, torch.Tensor]:
        z_all = self._encoder(x)
        z_mean, z_log_var = z_all[..., :self._latent_dim], z_all[..., self._latent_dim:]
        
        if sample:
            z_std = torch.sqrt(torch.exp(z_log_var))
            e = torch.randn_like(z_mean)
            z = z_mean + z_std * e
        else:
            z = z_mean
            
        x_hat = self._decoder(z)
        return {
            'x': x_hat,
            'z': z,
            'z_mean': z_mean,
            'z_log_var': z_log_var
        }
    
    def sample_from_prior(self) -> torch.Tensor:
        """
        Generates a sample from the prior distribution and decodes it into data space.

        Returns:
            The generated data sample.
        """
        device = next(self.parameters()).device
        z = torch.randn(self._latent_dim).to(device)
        return self._decoder(z)
    
    def sample_from_posterior(self, x_evidence: torch.Tensor) -> torch.Tensor:
        """
        Generates a sample from the posterior distribution conditioned on evidence and decodes it into data space.

        Args:
            x_evidence: The evidence to condition the posterior distribution on.

        Returns:
            The generated data sample.
        """
        output = self.forward(x_evidence, sample=True)
        return output['x']
    
vae = VariationalAutoEncoder()
vae.show_encoder_transforms(torch.randn(1, 28, 28, dtype=torch.float32))
vae.show_decoder_transforms(torch.randn(2, dtype=torch.float32))
output = vae(torch.randn(1, 28, 28, dtype=torch.float32))
{k: v.shape for k, v in output.items()}

#### Training VAE

The training process for the VAE tends to be more time-consuming than for the AE due to its inherent stochastic nature.

In [ ]:
class GaussianELBOLoss(nn.Module):
    """
    Computes the ELBO loss for a multivariate Gaussian distribution with a diagonal covariance matrix.
    """
    def __init__(self, noise: float):
        """
        Args:
            noise: Value of the covariance matrix, a constant.
                Increasing this value enhances the regularization effect.
        """
        super().__init__()
        self._noise = noise
        
    def forward(self, x_hat: torch.Tensor, x_target: torch.Tensor, z_mean: torch.Tensor, z_log_var: torch.Tensor) -> torch.Tensor:
        likelihood_loss = torch.mean(1 / (2 * self._noise) * torch.sum(torch.square(x_hat - x_target), dim=-1))
        z_var = torch.exp(z_log_var)
        kl_loss = torch.mean(- 1/2 * torch.sum(1 + z_log_var / 2 - z_var - torch.square(z_mean), dim=-1))
        return likelihood_loss + kl_loss, likelihood_loss, kl_loss

In [ ]:
def train_variational_autoencoder(
    model: VariationalAutoEncoder,
    train_dataset: Dataset,
    val_dataset: Dataset,
    epochs: int = 40,
    scheduler_step_size: int = 20,
    batch_size: int = 64,
    num_workers = 2,
    accelerator: str = 'cpu'
) -> dict:
    """
    Trains the Variational AutoEncoder (VAE) model and returns the training history.

    Args:
        model: The VariationalAutoEncoder model to be trained.
        train_dataset: The dataset used for training.
        val_dataset: The dataset used for validation during training.
        epochs: The number of epochs to train the model. Default is 20.
        scheduler_step_size: The number of epochs after which the scheduler decreases the learning rate. Default is 10.
        batch_size: The size of the batches used for training and validation. Default is 64.
        num_workers: The number of worker threads used with the DataLoader. Default is 2.
        accelerator: The device to perform operations. Default is 'cpu'. Options are 'cpu' or 'cuda' if GPU is available.

    Returns:
        A dictionary containing the training history, including training and validation loss per epoch.
    """
    loss_func = GaussianELBOLoss(noise=0.1)
    model.to(accelerator)
    optimizer = torch.optim.Adam(lr=1e-3, params=model.parameters())
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = scheduler_step_size)
    
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers)
    val_dataloader = DataLoader(val_dataset, batch_size, num_workers=num_workers, shuffle=True)
    
    history = defaultdict(list)
    
    for epoch in tqdm(range(1, epochs+1)):
        total_loss = 0.0
        total_likelihood_loss = 0.0
        total_kl_loss = 0.0
        n_steps = 0
        
        model.train()
        for img in train_dataloader:
            optimizer.zero_grad()
            
            img = img.to(accelerator)
            output = model(img)
            
            loss, likelihood_loss, kl_loss = loss_func(output['x'], img, output['z_mean'], output['z_log_var'])
            loss.backward()
            optimizer.step()
            
            total_loss += loss.detach().cpu().item()
            total_likelihood_loss += likelihood_loss.detach().cpu().item()
            total_kl_loss += kl_loss.detach().cpu().item()
            n_steps += 1
            
        scheduler.step()
            
        train_loss = total_loss / n_steps
        train_likelihood_loss = total_likelihood_loss / n_steps
        train_kl_loss = total_kl_loss / n_steps
        
        total_loss = 0.0
        total_likelihood_loss = 0.0
        total_kl_loss = 0.0
        n_steps = 0
        
        model.eval()
        with torch.no_grad():
            for img in val_dataloader:
                img = img.to(accelerator)
                output = model(img)

                loss, likelihood_loss, kl_loss = loss_func(output['x'], img, output['z_mean'], output['z_log_var'])
                
                total_loss += loss.detach().cpu().item()
                total_likelihood_loss += likelihood_loss.detach().cpu().item()
                total_kl_loss += kl_loss.detach().cpu().item()
                n_steps += 1
                
        val_loss = total_loss / n_steps
        c = total_likelihood_loss / n_steps
        val_kl_loss = total_kl_loss / n_steps
        
        print(f'[Epoch={epoch:02d}]: train_loss={train_loss:.2f}, val_loss={val_loss:.2f}')
        
        history['train_loss'].append(train_loss)
        history['train_likelihood_loss'].append(train_likelihood_loss)
        history['train_kl_loss'].append(train_kl_loss)
        history['val_loss'].append(val_loss)
        history['val_likelihood_loss'].append(val_kl_loss)
        history['val_kl_loss'].append(val_kl_loss)
        history['epoch'].append(epoch)
        
    return dict(history)
        

vae = VariationalAutoEncoder()
history = train_variational_autoencoder(
    model=vae,
    train_dataset=UnsupervisedMNIST(train=True),
    val_dataset=UnsupervisedMNIST(train=False),
    accelerator=ACCELERATOR
)

In [ ]:
def show_vae_training_history(history: dict) -> None:
    """
    Displays the training history of the Variational Autoencoder (VAE) model.

    Args:
        history: A dictionary containing the training history.
    """

    _, axs = plt.subplots(figsize=(14, 4), ncols=3)
    
    axs[0].plot(history['train_loss'], color='orange', label='train')
    axs[0].plot(history['val_loss'], color='blue', label='val')
    axs[0].set_xlabel('epoch')
    axs[0].set_ylabel('loss')
    axs[0].set_title('Loss History')
    axs[0].legend()
    
    axs[1].plot(history['train_likelihood_loss'], color='orange', label='train')
    axs[1].plot(history['val_likelihood_loss'], color='blue', label='val')
    axs[1].set_xlabel('epoch')
    axs[1].set_ylabel('loss')
    axs[1].set_title('Likelihood Loss History')
    axs[1].legend()
    
    axs[2].plot(history['train_kl_loss'], color='orange', label='train')
    axs[2].plot(history['val_kl_loss'], color='blue', label='val')
    axs[2].set_xlabel('epoch')
    axs[2].set_ylabel('loss')
    axs[2].set_title('KL Loss History')
    axs[2].legend()
    
    plt.show()
    
show_vae_training_history(history)

#### VAE demonstration

##### Reconstruction

In [ ]:
@torch.no_grad()
def show_naive_variational_autoencoder_error_analysis_visualization(
    model: VariationalAutoEncoder,
    val_dataset: Dataset,
    accelerator: str = 'cpu'
) -> None:
    """
    Displays a naive visualization for error analysis of a VAE.

    Args:
        model: The trained VAE model to use for image reconstruction.
        val_dataset: The dataset to use for generating the visualization. Typically,
            this should be a validation or test dataset.
        accelerator: The device to perform operations. Default is 'cpu'.
            Options are 'cpu' or 'cuda' if GPU is available.
    """
    N_SAMPLES = 6
    indices = list(range(len(val_dataset)))
    sampled_indices = random.choices(indices, k=N_SAMPLES)
    model.to(accelerator)
    
    _, axs = plt.subplots(figsize=(2 * N_SAMPLES + 1, 5), nrows=2, ncols=N_SAMPLES)
    for i in range(N_SAMPLES):
        index = sampled_indices[i]
        img = val_dataset[index]
        img = img.to(accelerator)
        output = model.forward(img)
        img_hat = output['x']
        img, img_hat = [v.detach().cpu().numpy() for v in [img, img_hat]]
        
        axs[0][i].imshow(img[0])
        axs[1][i].imshow(img_hat[0])
        axs[0][i].axis('off')
        axs[1][i].axis('off')
           
    plt.show()

show_naive_variational_autoencoder_error_analysis_visualization(
    model=vae,
    val_dataset=UnsupervisedMNIST(train=False),
    accelerator=ACCELERATOR
)

##### Latent space visualization

In [ ]:
@torch.no_grad()
def show_vae_map_latent_space_distribution_visualization(
    model: VariationalAutoEncoder,
    val_dataset: Dataset,
    accelerator: str = 'cpu'
) -> None:
    """
    Displays the distribution of data in the VAE latent space.

    Args:
        model: The VAE model.
        val_dataset: Dataset used for the visualization.
        accelerator: The device for calculations ('cpu' or 'cuda').
    """
    COLORS = ['red', 'blue', 'green', 'yellow', 'orange', 'cyan', 'gray', 'black', 'purple', 'pink']
    model.eval()
    model.to(accelerator)
    
    zs = defaultdict(list)
    for i in tqdm(range(len(val_dataset))):
        img = val_dataset[i].to(accelerator)
        label = val_dataset.label(i)
        output = model.forward(img, sample=False)
        z = output['z_mean']
        
        zs[label].append(z.detach().cpu())
        
    zs = {k: torch.stack(v) for k, v in zs.items()}
    zs = dict(sorted(zs.items()))
    plt.figure(figsize=(12, 8))
    for i, (label, z) in enumerate(zs.items()):
        z = z.numpy()
        plt.scatter(z[:, 0], z[:, 1], color=COLORS[i], label=str(label))
        
    plt.legend()
    plt.title('Latent Space')
    plt.xlabel('Z1')
    plt.ylabel('Z2')
    plt.show()
    
show_vae_map_latent_space_distribution_visualization(
    model=vae,
    val_dataset=UnsupervisedMNIST(train=False),
    accelerator=ACCELERATOR
)

##### Generating new samples (sampling from the prior)

In [ ]:
# Sample from prior
def visualize_vae_sampling_from_prior(
    model: VariationalAutoEncoder,
    accelerator: str = 'cpu'
) -> None:
    """
    Generates few samples from the VAE prior.

    Args:
        model: The VAE model.
        accelerator: The device for calculations ('cpu' or 'cuda').
    """
    model.to(accelerator)
    model.eval()
    
    _, axs = plt.subplots(figsize=(14, 10), nrows=5, ncols=5)
    for i in range(5):
        for j in range(5):
            ax = axs[i][j]
            img = vae.sample_from_prior()
            ax.imshow(img.detach().cpu().numpy()[0])
            ax.axis('off')
    plt.show()
    
visualize_vae_sampling_from_prior(
    model=vae,
    accelerator=ACCELERATOR
)

##### Generating new samples similar to the given one (sampling from the posterior)

In [ ]:
# Sample from posterior
def visualize_vae_sampling_from_posterior(
    model: VariationalAutoEncoder,
    evidence: torch.Tensor,
    accelerator: str = 'cpu'
) -> None:
    model.to(accelerator)
    model.eval()
    evidence = evidence.to(accelerator)
    
    _, axs = plt.subplots(figsize=(14, 10), nrows=5, ncols=5)
    for i in range(5):
        for j in range(5):
            ax = axs[i][j]
            img = vae.sample_from_posterior(evidence)
            ax.imshow(img.detach().cpu().numpy()[0])
            ax.axis('off')
    plt.show()
    
visualize_vae_sampling_from_posterior(
    model=vae,
    evidence=UnsupervisedMNIST(train=False)[7865],
    accelerator=ACCELERATOR
)

##### Interpolation over the latent space

In [ ]:
# Interpolation
def vizualize_vae_interpolation(
    model: VariationalAutoEncoder,
    accelerator: str = 'cpu'
) -> None:
    model.to(accelerator)
    model.eval()
    
    _, axs = plt.subplots(figsize=(16, 10), nrows=10, ncols=10)
    for i in range(10):
        for j in range(10):
            ax = axs[i][j]
            z = torch.tensor([-1 + 2 * (i / 10), -1 + 2 * (j / 10)], dtype=torch.float32)
            z = z.to(accelerator)
            img = vae._decoder(z)
            ax.imshow(img.detach().cpu().numpy()[0])
            ax.axis('off')
    plt.show()
    
vizualize_vae_interpolation(
    model=vae,
    accelerator=ACCELERATOR
)

### Training VAE on the Fashion MNIST dataset

In [ ]:
fashion_vae = VariationalAutoEncoder()
history = train_variational_autoencoder(
    model=vae,
    train_dataset=UnsupervisedMNIST(train=True, fashion=True),
    val_dataset=UnsupervisedMNIST(train=False, fashion=True),
    accelerator=ACCELERATOR
)
show_vae_training_history(history)

In [ ]:
vizualize_vae_interpolation(
    model=fashion_vae,
    accelerator=ACCELERATOR
)